In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import numpy as np
from math import log
from CIoTS import *
import json
from time import time
import random

/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
runs = 20
p_range = [5, 10, 15, 20] 
dimensions_range = [3, 5, 10]
in_edges_range = [2, 4, 6]
data_length = 10000
test_length = 2000
alpha = 0.05
auto_corr = 0.75
patience = 2*max(p_range)
p_max = 2*max(p_range)

result_dir = 'results/p_estimation/'

### PC1 with VAR for p estimation

with open(result_dir+'estimates_pc1.json', 'r') as f:
    results_var = json.load(f)

for setting in results_var:
    for experiment in setting['experiments']:
        del experiment['iterations']    
    
for setting in results_var:
    for experiment in setting['experiments']:
        experiment['estimates'] = []
        ts = pd.read_csv(result_dir+experiment['data'])
        ts_test = pd.read_csv(result_dir+experiment['test_data'])
        
        start_time = time()
        var_ranking, var_scores = var_order_select(ts, setting['p_max'], ['bic']) 
        end_time = time()
        
        for p_est, score in zip(var_ranking['bic'], var_scores['bic']):
            _, graphs, times, bics, _, _ = pc_incremental_pc1(partial_corr_test, ts, setting['alpha'], p_est+1, 
                                                              verbose=True, patiency=0, steps=p_est)
            # based on data
            test_var = VAR(p_est)
            test_var.fit(ts)
            var_test_mse, var_test_bic = test_var.evaluate_test_set(ts_test[:p_est], ts_test[p_est:])
            
            # based on graph
            test_var = VAR(p_est)
            nm, dm = transform_ts(ts, p_est)
            test_var.fit_from_graph(setting['dim'], dm, graphs[p_est], nm)
            test_mse, test_bic = test_var.evaluate_test_set(ts_test[:p_est], ts_test[p_est:])
            
            experiment['estimates'].append({'p_est': int(p_est), 'time_pc1': float(times[p_est]), 
                                            'time_var': int(end_time-start_time), 
                                            'graph': nx.to_dict_of_lists(graphs[p_est]),
                                            'bic': float(bics[p_est]), 'var_bic': float(score), 
                                            'test_bic': float(test_bic), 'var_test_bic': float(var_test_bic),
                                            'test_mse': float(test_mse), 'var_test_mse': float(var_test_mse)})
            
    incoming_edges = setting['in_edges']
    p = setting['p']
    print(f'{incoming_edges} {p} done')

    with open(result_dir+'estimates_var_pre2.json', 'w') as f:
        json.dump(results_var, f)

In [6]:
### PC1 with VAR for p estimation (no Graph)

with open(result_dir+'estimates_pc1.json', 'r') as f:
    results_var = json.load(f)

for setting in results_var:
    for experiment in setting['experiments']:
        del experiment['iterations']    
    
for setting in results_var:
    for experiment in setting['experiments']:
        experiment['estimates'] = []
        ts = pd.read_csv(result_dir+experiment['data'])
        ts_test = pd.read_csv(result_dir+experiment['test_data'])
        
        start_time = time()
        var_ranking, var_scores = var_order_select(ts, setting['p_max'], ['bic']) 
        end_time = time()
        
        for p_est, score in zip(var_ranking['bic'], var_scores['bic']):
            # based on data
            test_var = VAR(p_est)
            test_var.fit(ts)
            var_test_mse, var_test_bic = test_var.evaluate_test_set(ts_test[:p_est], ts_test[p_est:])
            
            experiment['estimates'].append({'p_est': int(p_est), 'time_pc1': 0, 
                                            'time_var': int(end_time-start_time), 
                                            'graph': {},
                                            'bic': float(score), 'test_bic': float(var_test_bic), 
                                            'test_mse': float(var_test_mse)})
            
    incoming_edges = setting['in_edges']
    p = setting['p']
    print(f'{incoming_edges} {p} done')

    with open(result_dir+'estimates_var_pre2.json', 'w') as f:
        json.dump(results_var, f)

2 5 done
2 5 done
2 5 done
2 10 done
2 10 done
2 10 done
2 15 done
2 15 done
2 15 done
2 20 done
2 20 done
2 20 done
4 5 done
4 5 done
